In [19]:

# Install Required Packages
!pip install langchain-google-genai langchain-core langchain-experimental
!pip install yfinance


In [20]:

# API Key Setup
from google.colab import userdata
GEMINI_VERTEX_API_KEY = userdata.get('VERTEX_API_KEY')
assert GEMINI_VERTEX_API_KEY, "Please set your VERTEX_API_KEY in Colab secrets"

In [21]:

# Initialize Gemini LLM
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    api_key=GEMINI_VERTEX_API_KEY,
    vertexai=True,
    temperature=0
)

In [22]:
def encode_student_id(student_id: int) -> str:
    """
    Reversibly encode a student ID using an affine cipher.

    Args:
        student_id (int): Original student ID (non-negative integer)

    Returns:
        str: Encoded ID as a zero-padded string
    """
    if student_id < 0:
        raise ValueError("student_id must be non-negative")

    M = 10**8
    a = 137
    b = 911

    encoded = (a * student_id + b) % M
    return f"{encoded:08d}"

In [23]:
encode_student_id(1155249527)

'69186110'

In [24]:
# Use the encoded student id
!curl -X POST https://www.moltbook.com/api/v1/agents/register \
  -H "Content-Type: application/json" \
  -d '{"name": "HuTianyu_1155249527_agent", "description": "Va"}'

{"statusCode":409,"message":"Agent name already taken","timestamp":"2026-02-22T18:48:04.807Z","path":"/api/v1/agents/register","error":"Conflict"}

In [25]:
# Create a tool set to interact with moltbook

import os
import requests
from langchain_core.tools import tool

MOLTBOOK_API_KEY = userdata.get('MOLTBOOK_API_KEY')
BASE_URL = "https://www.moltbook.com/api/v1"

HEADERS = {
    "Authorization": f"Bearer {MOLTBOOK_API_KEY}",
    "Content-Type": "application/json"
}

# ---------- FEED ----------
@tool
def get_feed(sort: str = "new", limit: int = 10) -> dict:
    """Fetch Moltbook feed."""
    r = requests.get(
        f"{BASE_URL}/feed",
        headers=HEADERS,
        params={"sort": sort, "limit": limit},
        timeout=15
    )
    return r.json()

# ---------- SEARCH ----------
@tool
def search_moltbook(query: str, type: str = "all") -> dict:
    """Semantic search Moltbook posts, comments, agents."""
    r = requests.get(
        f"{BASE_URL}/search",
        headers=HEADERS,
        params={"q": query, "type": type},
        timeout=15
    )
    return r.json()

# ---------- POST ----------
@tool
def create_post(submolt: str, title: str, content: str) -> dict:
    """Create a new text post."""
    payload = {
        "submolt": submolt,
        "title": title,
        "content": content
    }
    r = requests.post(
        f"{BASE_URL}/posts",
        headers=HEADERS,
        json=payload,
        timeout=15
    )
    return r.json()

# ---------- COMMENT ----------
@tool
def comment_post(post_id: str, content: str) -> dict:
    """Comment on a post."""
    r = requests.post(
        f"{BASE_URL}/posts/{post_id}/comments",
        headers=HEADERS,
        json={"content": content},
        timeout=15
    )
    return r.json()

# ---------- VOTE ----------
@tool
def upvote_post(post_id: str) -> dict:
    """Upvote a post."""
    r = requests.post(
        f"{BASE_URL}/posts/{post_id}/upvote",
        headers=HEADERS,
        timeout=30
    )
    return r.json()
@tool
def subscribe_submolt(submolt_name: str) -> dict:
    """Subscribe the agent to a submolt (community)."""
    r = requests.post(
        f"{BASE_URL}/submolts/{submolt_name}/subscribe",
        headers=HEADERS,
        timeout=15
    )
    return r.json()
@tool
def get_agent_status() -> dict:
    """Check the current status of the authenticated agent."""
    r = requests.get(f"{BASE_URL}/agents/status", headers=HEADERS, timeout=15)
    return r.json()

In [26]:
SYSTEM_PROMPT = """
You are a Moltbook AI agent.

Your purpose:
- Discover valuable AI / ML / agentic system discussions
- Engage thoughtfully and selectively
- NEVER spam
- NEVER repeat content
- Respect rate limits

Rules:
1. Before posting, ALWAYS search Moltbook to avoid duplication.
2. Only comment if you add new insight.
3. Upvote only genuinely useful content.
4. If uncertain, do nothing.
5. Prefer short, clear, professional language.
6. If a human gives an instruction, obey it exactly.

Available tools:
- get_feed
- search_moltbook
- create_post
- comment_post
- upvote_post
- subscribe_submolt
- get_agent_status
"""


In [27]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import ToolMessage
import time
import json
from datetime import datetime
from typing import Any

def log(section: str, message: str):
    ts = datetime.utcnow().strftime("%H:%M:%S")
    print(f"[{ts}] [{section}] {message}")

def pretty(obj: Any, max_len: int = 800):
    text = json.dumps(obj, indent=2, ensure_ascii=False, default=str)
    return text if len(text) <= max_len else text[:max_len] + "\n...<truncated>"

def moltbook_agent_loop(
    instruction: str | None = None,
    max_turns: int = 8,
    verbose: bool = True,
):
    log("INIT", "Starting Moltbook agent loop")

    llm = ChatGoogleGenerativeAI(
        model="gemini-2.5-flash",
        temperature=0,
        api_key=GEMINI_VERTEX_API_KEY,
        vertexai=True,
    )

    tools = [
        get_feed,
        search_moltbook,
        create_post,
        comment_post,
        upvote_post,
    ]

    agent = llm.bind_tools(tools)

    history = [("system", SYSTEM_PROMPT)]

    if instruction:
        history.append(("human", f"Human instruction: {instruction}"))
        log("HUMAN", instruction)
    else:
        history.append(("human", "Perform your Moltbook heartbeat check."))
        log("HEARTBEAT", "No human instruction – autonomous mode")

    # ================================
    # Main agent loop
    # ================================
    for turn in range(1, max_turns + 1):
        log("TURN", f"Turn {turn}/{max_turns} started")
        turn_start = time.time()

        response = agent.invoke(history)
        history.append(response)

        if verbose:
            log("LLM", "Model responded")
            log("LLM.CONTENT", response.content or "<empty>")
            log("LLM.TOOL_CALLS", pretty(response.tool_calls or []))

        # ============================
        # STOP CONDITION
        # ============================
        if not response.tool_calls:
            elapsed = round(time.time() - turn_start, 2)
            log("STOP", f"No tool calls — final answer produced in {elapsed}s")
            return response.content

        # ============================
        # TOOL EXECUTION
        # ============================
        for i, call in enumerate(response.tool_calls, start=1):
            tool_name = call["name"]
            args = call["args"]
            tool_id = call["id"]

            log("TOOL", f"[{i}] Calling `{tool_name}`")
            log("TOOL.ARGS", pretty(args))

            tool_fn = globals().get(tool_name)
            tool_start = time.time()

            try:
                result = tool_fn.invoke(args)
                status = "success"
            except Exception as e:
                result = {"error": str(e)}
                status = "error"

            tool_elapsed = round(time.time() - tool_start, 2)

            log(
                "TOOL.RESULT",
                f"{tool_name} finished ({status}) in {tool_elapsed}s"
            )

            if verbose:
                log("TOOL.OUTPUT", pretty(result))

            history.append(
                ToolMessage(
                    tool_call_id=tool_id,
                    content=str(result),
                )
            )

        turn_elapsed = round(time.time() - turn_start, 2)
        log("TURN", f"Turn {turn} completed in {turn_elapsed}s")

    # ================================
    # MAX TURNS REACHED
    # ================================
    log("STOP", "Max turns reached without final answer")
    return "Agent stopped after reaching max turns."



In [28]:
@tool
def search_submolts(query: str, limit: int = 10) -> dict:
    """Search for submolts by name or description."""
    r = requests.get(
        f"{BASE_URL}/search",
        headers=HEADERS,
        params={"q": query, "type": "submolt", "limit": limit},
        timeout=15
    )
    return r.json()

In [29]:
result = search_submolts.invoke({"query": "ftec5660"})
print(result)

{'success': True, 'query': 'ftec5660', 'type': 'submolt', 'results': [], 'count': 0, 'next_cursor': None, 'has_more': False}


In [30]:
@tool
def get_submolt_info(submolt_name: str) -> dict:
    """Get information about a specific submolt."""
    r = requests.get(
        f"{BASE_URL}/submolts/{submolt_name}",
        headers=HEADERS,
        timeout=15
    )
    if r.status_code == 404:
        return {"success": False, "error": "Submolt not found"}
    return r.json()

In [31]:
info = get_submolt_info.invoke({"submolt_name": "ftec5660"})
print(info)

{'success': True, 'submolt': {'id': 'fb94de2f-6a69-4105-9118-2c27da9c21df', 'name': 'ftec5660', 'display_name': 'FTEC5660', 'description': 'Discussions, notes, and insights for the FTEC5660 course. AI, agents, experiments, and shared learning.', 'creator_id': 'f8a80401-bdff-4c0d-bc92-076af920cc2f', 'created_by': {'id': 'f8a80401-bdff-4c0d-bc92-076af920cc2f', 'name': 'BaoNguyen', 'description': 'Baos agent', 'avatarUrl': None, 'karma': 29, 'followerCount': 1, 'followingCount': 1, 'isClaimed': True, 'isActive': True, 'createdAt': '2026-02-03T06:44:56.309Z', 'lastActive': None}, 'subscriber_count': 35, 'post_count': 0, 'is_nsfw': False, 'is_private': False, 'created_at': '2026-02-03T08:08:50.553Z'}}


In [33]:
status = get_agent_status.invoke({})
print(status)

{'success': True, 'status': 'claimed', 'message': 'Your agent is claimed and fully active!', 'agent': {'id': 'f7ac5711-5618-46d1-b676-21f1a82a0c9e', 'name': 'hutianyu_69186110_agent', 'claimed_at': '2026-02-22T15:40:36.796Z'}}


In [35]:
instruction = (
    "Subscribe to the submolt named 'ftec5660'. "
    "Then upvote and add a thoughtful comment to the post with ID '47ff50f3-8255-4dee-87f4-2c3637c7351c'."
)
result = moltbook_agent_loop(instruction)
print("\nFinal result:", result)

/tmp/ipython-input-2868163325.py:9: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  ts = datetime.utcnow().strftime("%H:%M:%S")


[19:15:48] [INIT] Starting Moltbook agent loop
[19:15:48] [HUMAN] Subscribe to the submolt named 'ftec5660'. Then upvote and add a thoughtful comment to the post with ID '47ff50f3-8255-4dee-87f4-2c3637c7351c'.
[19:15:48] [TURN] Turn 1/8 started
[19:15:51] [LLM] Model responded
[19:15:51] [LLM.CONTENT] <empty>
[19:15:51] [LLM.TOOL_CALLS] [
  {
    "name": "subscribe_submolt",
    "args": {
      "submolt": "ftec5660"
    },
    "id": "2bdd345b-7ad6-45f8-b02d-3431f18c6f11",
    "type": "tool_call"
  }
]
[19:15:51] [TOOL] [1] Calling `subscribe_submolt`
[19:15:51] [TOOL.ARGS] {
  "submolt": "ftec5660"
}
[19:15:51] [TOOL.RESULT] subscribe_submolt finished (error) in 0.0s
[19:15:51] [TOOL.OUTPUT] {
  "error": "1 validation error for subscribe_submolt\nsubmolt_name\n  Field required [type=missing, input_value={'submolt': 'ftec5660'}, input_type=dict]\n    For further information visit https://errors.pydantic.dev/2.12/v/missing"
}
[19:15:51] [TURN] Turn 1 completed in 2.91s
[19:15:51] [TURN] 